#PyMuPDFLoader

In [ ]:
!pip install pycoingecko requests tiktoken cohere openai pinecone-client langchain apify-client langchainhub pymupdf

###1.IMPORT LIBRARY

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import PyMuPDFLoader
import numpy as np
import datetime as dt
import openai
import re
import time
import pinecone
from langchain.document_loaders import TextLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Pinecone

###2.KEY SETTINGS

In [ ]:
import os
from langchain.vectorstores import Vectara

os.environ["OPENAI_API_KEY"]      = "OPENAI_API_KEY"
os.environ["VECTARA_CUSTOMER_ID"] = "VECTARA_CUSTOMER_ID"
os.environ["VECTARA_CORPUS_ID"]   = "VECTARA_CORPUS_ID"
os.environ["VECTARA_API_KEY"]     = "VECTARA_API_KEY"

###3.INITIALIZE VECTARA

In [ ]:
vectara = Vectara(
    vectara_customer_id = os.getenv("VECTARA_CUSTOMER_ID"),
    vectara_corpus_id   = os.getenv("VECTARA_CORPUS_ID"),
    vectara_api_key     = os.getenv("VECTARA_API_KEY")
)

###4.Upload Document

In [ ]:
loader = PyMuPDFLoader("/content/FILE")

In [ ]:
documents = loader.load()

###4-1.metadata

In [ ]:
documents[0].metadata

In [ ]:
vectorstore = Vectara.from_documents(documents, embedding=None, doc_metadata={"doc_type":"pdf", "doc_content":"semicondoctor"}
)

In [ ]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

In [ ]:
retriever = vectorstore.as_retriever(lambda_val=0.025, k=5, filter=None)

In [ ]:
# langchain hub 에서 Prompt 다운로드 예시
# https://smith.langchain.com/hub/rlm/rag-prompt

from langchain import hub

rag_prompt = hub.pull("rlm/rag-prompt")
rag_prompt

In [ ]:
# LLM
from langchain.chat_models import ChatOpenAI

# ChatGPT 모델 지정
llm = ChatOpenAI(model_name="gpt-3.5-turbo-1106", temperature=0)
#llm = ChatOpenAI(model_name='gpt-3.5-turbo-1106',temperature=0)

In [ ]:
# RAG chain 생성
from langchain.schema.runnable import RunnablePassthrough

# pipe operator를 활용한 체인 생성
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | rag_prompt
    | llm
)

In [ ]:
rag_chain.invoke("반도체 수출에 가장 큰 영향을 미치는 요인은 무엇인가요?")

In [ ]:
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI

search_kwargs = {
    "filter":{"doc.doc_content = 'semicondoctor"}

}
qa_filter = RetrievalQA.from_llm(llm=llm, retriever=vectara.as_retriever(search_type="similarity",search_kwargs=search_kwargs))
qa_filter({"query":"신용도판단", })

In [ ]:
rag_chain.invoke("반도체 수출 회복에 대한 기대감은 어떻게 형성되나요?")

In [ ]:
rag_chain.invoke("반도체 수출에 가장 큰 영향을 미치는 요인은 무엇인가요?")